In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
import json
import math
import csv
import pandas as pd
from scipy import optimize
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
#define architecture for three waveforms
class Net(nn.Module):                        # model class      
    def __init__(self):                      
        super().__init__()
        self.fc1 = nn.Linear(12, 16)    # four-layered network; number of neurons in each layer
        self.fc2 = nn.Linear(16, 256)   # equals number of output; no. of bias = no. of outputs; weights = no. of(input*output)
        self.fc3 = nn.Linear(256, 16)
        self.fc4 = nn.Linear(16,1)


    def forward(self, x):              # x is the input vector #
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)                # real value
        return x

In [2]:
data = pd.read_excel('./DataSplit/'+Material+'/_Trap/' + Material +'_Trap'+ '.xlsx')

data = data[data['D1']!=0]
data = data[data['D3']!=0]

DataF = data['Freq (kHz)'].to_numpy()
DataB = data['Bpk (mT)'].to_numpy()
DataT = data['T'].to_numpy()
DataD1 = data['D1'].to_numpy()
DataD2 = data['D2'].to_numpy()
DataD3 = data['D3'].to_numpy()
CritB1 = data['CritB1 (mT)'].to_numpy()
CritB2 = data['CritB2 (mT)'].to_numpy()
CritB3 = data['CritB3 (mT)'].to_numpy()
CritB4 = data['CritB4 (mT)'].to_numpy()
DataIndTrap = data['DataIndex'].to_numpy() 

f1 = DataF/DataD1
f2 = DataF/DataD2
f3 = DataF/DataD3
B1 = np.abs(CritB1 - CritB4)/2
B2 = np.abs(CritB2 - CritB1)/2
for i in range(len(B2)):
    if B2[i] == 0:
        B2[i] = np.abs(CritB2[i])
        f2[i] = 1
B3 = np.abs(CritB3 - CritB2)/2

f1 = np.log10(f1)
f2 = np.log10(f2)
f3 = np.log10(f3)
B1 = np.log10(B1)
B2 = np.log10(B2)
B3 = np.log10(B3)
f  = np.log10(DataF)
DataT = np.log10(DataT)
DataB = np.log10(DataB)


DataD1 = DataD1.reshape((-1,1))
DataD2 = DataD2.reshape((-1,1))
DataD3 = DataD3.reshape((-1,1))
f = f.reshape((-1,1))
f1 = f1.reshape((-1,1))
f2 = f2.reshape((-1,1))
f3 = f3.reshape((-1,1))
B1 = B1.reshape((-1,1))
B2 = B2.reshape((-1,1))
B3 = B3.reshape((-1,1))
DataT = DataT.reshape((-1,1))
DataB = DataB.reshape((-1,1))

input_data = np.concatenate((f1,f2,f3,B1,B2,B3,DataD1,DataD2,DataD3,DataT,f,DataB),axis=1)
input_tensors = torch.from_numpy(input_data).view(-1,12)
dataset_trap = torch.utils.data.TensorDataset(input_tensors)

C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:12: RuntimeWarning: invalid value encountered in log10
  DataF = np.log10(DataF1)
C:\Users\himan\AppData\Local\Temp\ipykernel_33336\3732659971.py:18: RuntimeWarning: invalid value encountered in log10
  DataTime = np.log10(DataTime1)


In [ ]:
test_loader_trap = torch.utils.data.DataLoader(dataset_trap, batch_size = 128, shuffle = False)

net_trap = Net().double()
ckpt_trap = torch.load('./Trained_Models/'+Material+'_trap_train.pt')
net_trap = ckpt_trap['model'] # load the trained model
y_pred = []
net_trap.eval()
with torch.no_grad():
    for inputs, in test_loader_trap:
        test_out = net_trap(inputs)
        y_pred.append(test_out)

y_pred = torch.cat(y_pred, dim=0)
trap_test_out = (10**(y_pred.numpy()))*1e3
trap_index    = DataIndTrap